In [1]:
import sys 
sys.path.append('../fsp')
import monteCarloSimulation as mcs
import bapm_exact as be
import numpy as np

In [2]:
# Example from the notes
S0 = 4
K = 5
u = 2
d = 0.5
r = 0.25

exp = be.bapm_exact(u, d, r, S0)
exp.setOption(True, True, K, 1)
assert(exp.stockPrice(3, 0) == S0 * d **3)
assert(exp.stockPrice(0, 3) == S0 * u **3)

In [3]:
V0 = exp.pathIndependentExpectedValue(2,exp.riskNeutralProbability())
print("V0: ", V0)


V0:  (array([1.76]), 4.000000000000001)


In [4]:
V0 = exp.singleStepReplicatingPortfolio(0, 0)
print("V0: ", V0)

V0:  [ 0. -0.]


In [5]:
maxPathLength = 4
exp.generateAllPaths(2)

[[0, 0], [0, 1], [1, 0], [1, 1]]

In [6]:
exp.recursiveReplicatingPortfolio(3)
exp.replicatingDeltas

[array([0.8]),
 array([0.2]),
 array([1.8]),
 array([-0.]),
 array([0.5]),
 array([0.5]),
 array([4.]),
 None]

The deltas are not path independent. 

In [7]:
# Example from the notes
S0 = 100
u = 1.1
d = 1.01
r = 0.05
N = 5
K = (1+r)**N * S0


exp = be.bapm_exact(u, d, r, S0)
exp.setOption(True, True, K, N)
p_tilda = exp.riskNeutralProbability()
V0 = exp.pathIndependentExpectedValue(N, p_tilda)
print("V0: ", V0[0], "SN: ", V0[1])

V0 = exp.pathIndependentExpectedValue(N, p_tilda - 0.2)
print("V0: ", V0[0], "SN: ", V0[1])

V0 = exp.pathIndependentExpectedValue(N, p_tilda + 0.2)
print("V0: ", V0[0], "SN: ", V0[1])

V0 = exp.recursiveReplicatingPortfolio(N)
print("V0: ", V0[0], "Deltas (in a binary tree): ", V0[1])


V0:  [3.90314752] SN:  99.99999999999996
V0:  [0.76909469] SN:  91.71745411267206
V0:  [9.93496062] SN:  108.87038735338368
V0:  [3.90314752] Deltas (in a binary tree):  [array([0.53518931]), array([0.35939113]), array([0.81514582]), array([0.16218839]), array([0.64632605]), array([0.64632605]), array([1.11787445]), array([-0.]), array([0.38317008]), array([0.38317008]), array([1.0479827]), array([0.38317008]), array([1.0479827]), array([1.0479827]), array([1.331]), array([-0.]), array([-0.]), array([-0.]), array([0.90523931]), array([-0.]), array([0.90523931]), array([0.90523931]), array([1.34431]), array([-0.]), array([0.90523931]), array([0.90523931]), array([1.34431]), array([0.90523931]), array([1.34431]), array([1.34431]), array([1.4641]), None]


Monte Carlo


In [8]:
S0 = 100
u = 1.1
d = 1.01
r = 0.05
N = 5
K = (1+r)**N * S0


mc = mcs.MonteCarlo(up_factor=u, down_factor=d, risk_free_rate=r, S0=100)
mc.setOption(True, True, K, N)
mc.setRNG(np.random.seed(22))
print(p_tilda)
S, V = mc.simulatePathDependent(100, 5, 3200000, p_tilda)
print("V0: ", np.mean(V), "SN: ", np.mean(S[1]))


0.4444444444444444
0.44426775
V0:  3.8979751820437403 SN:  115.86322669297357
